## Ejercicio unión de bases de datos

## Contexto:
---

Los fabricantes de grandes máquinas analizan rutinariamente los materiales que utilizan. Este conjunto de datos contiene información sobre varios tubos utilizados en máquinas. Puedes imaginar que las grandes máquinas requieren diferentes tipos de tubos; los coches tienen tubos, los aviones tienen tubos y las excavadoras también tienen tubos. Hablemos un poco sobre cómo se ve un tubo en este conjunto de datos.

La parte más pequeña de un ensamblaje de tubos es un componente. Los componentes pueden ser cosas como abrazaderas, juntas o algún otro tipo de accesorios. En general, los ensamblajes de tubos consisten en uno o más componentes y también pueden ser diferentes combinaciones de componentes. En este ejercicio, analizarás los datos y responderás algunas preguntas sobre los pesos. Sin embargo, antes de poder analizar los datos, debes organizarlos en un formato manejable. Necesitarás usar lo que has aprendido sobre la combinación y agrupación de datos.

Tienes estos dos dataframes para responder las preguntas. Echa un vistazo y familiarízate con ellos:

- **component_counts**: contiene la cantidad de componentes en cada ensamblaje de tubos.
- **component_weights**: contiene los pesos de los componentes individuales.

### Preguntas a contestar 

1. Cuales son los 5 tubos más pesados? cuales son sus pesos?
2. Cuantos *tube_assembly_id* tienen más de cinco componentes? R: 120
3. Cuantos *component_id* son usados en más de 50 *tube_assemblies*? R:69
4. Cual es el peso promedio de los 5 componenentes más pesados? R: 95.4894

### Referencias
- https://github.com/timmyshen/Cat_Tube
- https://www.kaggle.com/code/crawford/python-groupby-and-merge-exercises/notebook


## Ejercicio
---
En este ejercicio, nos adentraremos en el análisis de datos relacionados con la fabricación de grandes máquinas, centrando nuestra atención en los tubos que las componen. El **objetivo** de este análisis es responder preguntas específicas sobre los pesos de estos tubos y sus componentes. Para ello, trabajaremos con dos dataframes clave: `component_counts`, que registra la cantidad de componentes por ensamblaje de tubos, y `component_weights`, que contiene los pesos individuales de los componentes.

Nuestro enfoque se centrará en combinar y agrupar estos datos para obtener insights valiosos, como identificar los tubos más pesados y analizar la distribución de componentes en los ensamblajes.

El primer paso será importar el ambiente y las dos distintas bases de datos. 

In [36]:
# Set del ambiente e importación del dataframe

## Importar paquetes
import pandas as pd

## Cargar base de datos
### Número de componentes
component_counts = pd.read_csv("../exercises/data/component_count.csv")

### Peso componentes
component_weights = pd.read_csv("../exercises/data/component_weights.csv")

### Inspección de los datos
Antes de realizar el análisis, es necesario realizar una inspección inicial de los datos. Esto nos permitirá entender mejor la estructura de los dataframes, identificar cualquier valor nulo o anómalo, y verificar la consistencia entre las columnas clave. Durante esta inspección,analizaremos las dimensiones.


In [37]:
# Información general de la tabla de conteo de componentes
## Información general 
component_counts.info()
print()

## Inspección general 
component_counts.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39459 entries, 0 to 39458
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   tube_assembly_id  39459 non-null  object 
 1   component_id      39459 non-null  object 
 2   component_count   39459 non-null  float64
dtypes: float64(1), object(2)
memory usage: 924.9+ KB



,tube_assembly_id,component_id,component_count
19905,TA-01097,C-0449,2.0
28156,TA-12550,C-1628,1.0
27013,TA-10992,C-1846,2.0
30121,TA-15287,C-1660,1.0
15544,TA-17261,C-1845,1.0


In [38]:
# Información general de la tabla de peso de los componentes
## Información general 
component_weights.info()
print()

## Inspección general 
component_weights.sample(5)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2047 entries, 0 to 2046
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   component_id  2047 non-null   object 
 1   weight        2047 non-null   float64
dtypes: float64(1), object(1)
memory usage: 32.1+ KB



,component_id,weight
593,C-1013,0.364
1563,C-0434,0.191
1694,C-1392,0.077
1162,C-1003,0.000
497,C-1502,0.418


In [39]:
## Inspección valores nulos (Confirmación)
### Para número de componentes
print("Valores ausentes tabla número de componentes: ", component_counts.isna().sum().sum())

### Para peso componentes
print("Valores ausentes tabla peso de componentes: ", component_weights.isna().sum().sum())

Valores ausentes tabla número de componentes:  0
Valores ausentes tabla peso de componentes:  0


In [40]:
## Inspección valores duplicados

### Para número de componentes
print(component_counts["component_id"].value_counts())
print("Valores únivocos en tabla de número de componentes: ", component_counts["component_id"].nunique())
print("Valores duplicados en tabla de número de componentes: ", component_counts["component_id"].duplicated().sum())
print()

### Para peso de componentes 
print(component_weights["component_id"].value_counts())
print("Valores únivocos en tabla de peso de componentes: ", component_weights["component_id"].nunique())
print("Valores duplicados en tabla de peso de componentes: ", component_weights["component_id"].duplicated().sum())


component_id
C-1621    2114
C-1628    2103
C-1629    1965
C-1622    1933
C-1624    1585
          ... 
C-1049       1
C-0981       1
C-0967       1
C-0934       1
C-0663       1
Name: count, Length: 2048, dtype: int64
Valores únivocos en tabla de número de componentes:  2048
Valores duplicados en tabla de número de componentes:  37411

component_id
C-2033    1
C-0005    1
C-0006    1
C-1435    1
C-1546    1
         ..
C-1229    1
C-1198    1
C-1197    1
C-1196    1
C-1195    1
Name: count, Length: 2047, dtype: int64
Valores únivocos en tabla de peso de componentes:  2047
Valores duplicados en tabla de peso de componentes:  0


La inspección de la tabla `component_counts` muestra que cuenta con 39459 observaciones sin valores nulos. Dentro de estas observaciones, se identificaron 2048 `component_id` distintos. Aunque no existen duplicados explícitos en términos de filas idénticas, algunos `component_id` se repiten con frecuencia. Un ejemplo es el `component_id` "C-1621", que aparece 2,114 veces.

Por otro lado, la tabla `component_weights` consta de 2047 observaciones, sin valores nulos y sin duplicados.

Estas tablas están relacionadas mediante la columna `component_id`. Aunque hay una gran cantidad de `component_id`, podemos inferir que no hay variaciones de peso para un mismo componente, lo cual simplificará el análisis posterior.

### Manipulación de los datos
En esta etapa, nos centramos en la manipulación de los datos mediante la unión de las dos tablas: `component_counts` y `component_weights`,  ambas relacionadas por la columna component_id. Como resultado de esta unión tendrémos una nueva tabla `component_df`, facilitando el análisis posterior sobre los pesos de los tubos y sus componentes.

In [51]:
## Unión de las bases de datos
### Creación de la base de datos nueva
component_df = pd.merge(left=component_weights, right= component_counts, how = "inner", on="component_id")

print(component_df.head())

print(component_df.shape)

component_df2 = pd.merge(left=component_counts, right= component_weights, how = "outer", on="component_id")
print(component_df2.shape)
print(component_df2.head())


  component_id  weight tube_assembly_id  component_count
0       C-0005   0.206         TA-01388              1.0
1       C-0006   0.083         TA-01219              1.0
2       C-1435   0.023         TA-00695              2.0
3       C-1435   0.023         TA-00696              2.0
4       C-1435   0.023         TA-00832              2.0
(39457, 4)
(39459, 4)
  tube_assembly_id component_id  component_count  weight
0         TA-21143         9999              1.0     NaN
1         TA-21144         9999              1.0     NaN
2         TA-00416       C-0001              2.0   0.013
3         TA-00417       C-0001              2.0   0.013
4         TA-00602       C-0001              2.0   0.013
